In [21]:
import pandas as pd

In [22]:
df = pd.read_csv('/content/WELFake_Dataset.csv')

In [23]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [58]:
df['text'][3]

'A dozen politically active pastors came here for a private dinner Friday night to hear a conversion story unique in the context of presidential politics: how Louisiana Gov. Bobby Jindal traveled from Hinduism to Protestant Christianity and, ultimately, became what he calls an “evangelical Catholic.”\n\nOver two hours, Jindal, 42, recalled talking with a girl in high school who wanted to “save my soul,” reading the Bible in a closet so his parents would not see him and feeling a stir while watching a movie during his senior year that depicted Jesus on the cross.\n\n“I was struck, and struck hard,” Jindal told the pastors. “This was the Son of God, and He had died for our sins.”\n\nJindal’s session with the Christian clergy, who lead congregations in the early presidential battleground states of Iowa and South Carolina, was part of a behind-the-scenes effort by the Louisiana governor to find a political base that could help propel him into the top tier of Republican candidates seeking t

In [24]:
df=df.drop('Unnamed: 0',axis=1)

In [25]:
df.head()

,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,NaN,Did they post their votes for Hillary already?,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [26]:
df.dropna(inplace=True)

In [27]:
X = df.drop('label',axis=1)
y = df['label']

In [28]:
X.isnull().sum()

,0
title,0
text,0


In [29]:
X.shape,y.shape

((71537, 2), (71537,))

In [30]:
y.value_counts()

,count
label,
1,36509
0,35028


In [33]:
!pip install tensorflow
import tensorflow as tf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 803.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 104.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 153.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.3 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [34]:
tf.__version__

'2.20.0'

In [35]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [36]:
voc_size=5000

In [37]:
messages=X.copy()

In [38]:
messages.head(10)

,title,text
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ..."
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will..."
5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...
6,DR BEN CARSON TARGETED BY THE IRS: “I never ha...,DR. BEN CARSON TELLS THE STORY OF WHAT HAPPENE...
7,HOUSE INTEL CHAIR On Trump-Russia Fake Story: ...,
8,Sports Bar Owner Bans NFL Games…Will Show Only...,"The owner of the Ringling Bar, located south o..."
9,Latest Pipeline Leak Underscores Dangers Of Da...,"FILE – In this Sept. 15, 2005 file photo, the ..."
10,GOP Senator Just Smacked Down The Most Puncha...,The most punchable Alt-Right Nazi on the inter...


In [39]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [40]:
messages.reset_index(inplace=True)

In [41]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
  review = re.sub('[^A-Za-z]',' ',messages['title'][i])
  review  = review.lower()
  review = review.split()
  review = [ps.stem(word)for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [42]:
corpus

['law enforc high alert follow threat cop white blacklivesmatt fyf terrorist video',
 'unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video',
 'bobbi jindal rais hindu use stori christian convers woo evangel potenti bid',
 'satan russia unv imag terrifi new supernuk western world take notic',
 'time christian group sue amazon splc design hate group',
 'dr ben carson target ir never audit spoke nation prayer breakfast',
 'hous intel chair trump russia fake stori evid anyth video',
 'sport bar owner ban nfl game show true american sport like speak rural america video',
 'latest pipelin leak underscor danger dakota access pipelin',
 'gop senat smack punchabl alt right nazi internet',
 'may brexit offer would hurt cost eu citizen eu parliament',
 'schumer call trump appoint offici overse puerto rico relief',
 'watch hilari ad call question health age clinton crime famili boss',
 'chang expect espn polit agenda despit huge subscrib declin breitbart'

In [43]:
onehot_rep = [one_hot(words,voc_size)for words in corpus]
onehot_rep

[[1872, 718, 3184, 4562, 1233, 1818, 1559, 4391, 3531, 260, 862, 2657],
 [3686,
  316,
  1475,
  2849,
  994,
  3762,
  886,
  612,
  3739,
  4386,
  3591,
  3272,
  1497,
  2657],
 [719, 3532, 3997, 42, 1791, 2152, 3581, 4375, 2150, 351, 330, 3520],
 [3116, 3772, 3009, 3776, 617, 4667, 558, 1530, 2115, 480, 1448],
 [1757, 3581, 113, 1363, 1188, 4444, 3464, 4077, 113],
 [4208, 2858, 244, 189, 2442, 2646, 3926, 1149, 4672, 3927, 1276],
 [1410, 2152, 3171, 1167, 3772, 1886, 2152, 1542, 2348, 2657],
 [3761,
  2905,
  507,
  3042,
  2685,
  2702,
  1156,
  2737,
  2773,
  3761,
  3921,
  3563,
  4764,
  640,
  2657],
 [4621, 1144, 1365, 1283, 543, 2302, 4654, 1144],
 [1834, 2718, 1868, 1415, 4904, 3397, 4930, 2820],
 [2099, 558, 4949, 673, 1020, 1516, 1253, 625, 1253, 4795],
 [1728, 2971, 1167, 4506, 442, 3484, 1992, 440, 3648],
 [3477, 2849, 4673, 2971, 3995, 4232, 2101, 1738, 4394, 4582, 1565],
 [144, 3391, 2162, 2616, 4203, 3824, 2122, 1601, 433, 131],
 [4651, 3948, 1543, 816, 1726, 141

In [44]:
sent_lenth = 20
embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=sent_lenth)
print(embedded_docs)

[[   0    0    0 ...  260  862 2657]
 [   0    0    0 ... 3272 1497 2657]
 [   0    0    0 ...  351  330 3520]
 ...
 [   0    0    0 ...  817 1571 1494]
 [   0    0    0 ...  413 2010 1587]
 [   0    0    0 ... 2266 1738 4841]]


In [47]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.build((None,sent_lenth))
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 20, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 200)            │       112,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 313,001 (1.19 MB)

 Trainable params: 313,001 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

In [49]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [50]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

In [51]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 22s 11ms/step - accuracy: 0.8310 - loss: 0.3630 - val_accuracy: 0.8943 - val_loss: 0.2506
Epoch 2/10
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.9133 - loss: 0.2130 - val_accuracy: 0.8914 - val_loss: 0.2564
Epoch 3/10
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 19s 11ms/step - accuracy: 0.9324 - loss: 0.1724 - val_accuracy: 0.8973 - val_loss: 0.2484
Epoch 4/10
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 19s 11ms/step - accuracy: 0.9452 - loss: 0.1395 - val_accuracy: 0.8972 - val_loss: 0.2653
Epoch 5/10
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 19s 11ms/step - accuracy: 0.9557 - loss: 0.1162 - val_accuracy: 0.8984 - val_loss: 0.2906
Epoch 6/10
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.9640 - loss: 0.0980 - val_accuracy: 0.8972 - val_loss: 0.3286
Epoch 7/10
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.9700 - loss: 0.0783 - val_accuracy: 0.8937 - val_loss: 0.3796
Epoch 8/10
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 19s 11ms/step - accuracy: 0.9762 -

In [52]:
y_pred=model.predict(X_test)

448/448 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


In [53]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [54]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[6266,  815],
       [ 703, 6524]])

In [55]:
print(accuracy_score(y_test,y_pred))

0.8939055074084429


In [61]:
# Sample data for testing (using a DataFrame with 'id', 'title', and 'text' columns)
sample_data_df = pd.DataFrame({
    'title': ["This is a fake news article about politics.",
              "The stock market is performing well today.",
              "Breaking news: Scientists discover new planet.",
              "This headline is completely made up and misleading.",
              "Bobby Jindal, raised Hindu, uses story of Christian conversion to woo evangelicals for potential 2016 bid"],
    'text': ["Some fake text.",
             "Some real text about the market.",
             "Details about the scientific discovery.",
             "More misleading text.",
             "Louisiana Governor Bobby Jindal shared his faith journey with pastors in Lynchburg, describing how he converted from Hinduism to Christianity after reading the Bible secretly in high school and being moved by a film about Jesus. Seeking support from evangelical conservatives for a potential 2016 presidential run, Jindal highlighted his defense of religious freedom and figures like Phil Robertson and Hobby Lobby. Speaking at Liberty University, he urged Christians to stand firm in their beliefs. Born to Indian immigrants, Jindal said his conversion was gradual and deeply personal, calling himself an “evangelical Catholic” committed to faith-driven leadership."]
})

# Preprocess the sample data (using the 'title' column)
def preprocess_text(text):
    review = re.sub('[^A-Za-z]', ' ', text)
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    return review

preprocessed_sample_titles = [preprocess_text(title) for title in sample_data_df['title']]

# One-hot encode and pad the preprocessed sample data
onehot_sample_rep = [one_hot(words, voc_size) for words in preprocessed_sample_titles]
embedded_sample_docs = pad_sequences(onehot_sample_rep, padding='pre', maxlen=sent_lenth)

# Predict the labels
predictions = model.predict(embedded_sample_docs)
predictions = np.where(predictions >= 0.5, 1, 0)

# Display the results
for i, title in enumerate(sample_data_df['title']):
    print(f"Title: {title}")
    print(f"Predicted label: {'Fake' if predictions[i][0] == 1 else 'Real'}")
    print("-" * 20)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Title: This is a fake news article about politics.
Predicted label: Fake
--------------------
Title: The stock market is performing well today.
Predicted label: Fake
--------------------
Title: Breaking news: Scientists discover new planet.
Predicted label: Fake
--------------------
Title: This headline is completely made up and misleading.
Predicted label: Fake
--------------------
Title: Bobby Jindal, raised Hindu, uses story of Christian conversion to woo evangelicals for potential 2016 bid
Predicted label: Real
--------------------
